In [1]:
# Importando as bibliotecas que serão usadas

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import awswrangler as wr

ModuleNotFoundError: No module named 'awswrangler'

In [2]:
# Importando a base de dados do S3

#df = wr.s3.read_csv(
#  path='s3://dataset_workshop.CSV',
#  sep=';',
#  encoding='latin-1'
#)


df = pd.read_csv('../Database/dataset_workshop.csv')

print(f"shape dos dados: {df.shape}")

df.head(10)

shape dos dados: (499, 1)


,Call Id;Date;Agent;Department;_Answered (Y/N);_Resolved;_Speed of Answer;_AvgTalkDuration;_Satisfaction rating;dur_aux;Answered (Y/N);Resolved;Speed of Answer;AvgTalkDuration;Satisfaction rating;_Satisfa��o;;;N;
0,ID0006;01/01/2015 11:57;Becky;Fridge;N;N;;;;;N...
1,ID0014;02/01/2015 13:45;Becky;Air Conditioner;...
2,ID0027;04/01/2015 14:49;Becky;Air Conditioner;...
3,ID0034;06/01/2015 11:12;Becky;Television;N;N;;...
4,ID0036;06/01/2015 17:21;Becky;Washing Machine;...
5,ID0039;07/01/2015 10:48;Becky;Television;N;N;;...
6,ID0041;07/01/2015 13:30;Becky;Television;N;N;;...
7,ID0046;08/01/2015 13:17;Becky;Toaster;N;N;;;;;...
8,ID0053;09/01/2015 13:39;Becky;Television;N;N;;...
9,ID0056;10/01/2015 11:47;Becky;Fridge;N;N;;;;;N...


In [3]:
#leitura direto da pasta - usando VSCode
df = pd.read_csv('../Database/dataset_workshop.csv', sep=';')

In [4]:
pd.set_option('display.max_columns', None)
df.head()

,Call Id,Date,Agent,Department,_Answered (Y/N),_Resolved,_Speed of Answer,_AvgTalkDuration,_Satisfaction rating,dur_aux,Answered (Y/N),Resolved,Speed of Answer,AvgTalkDuration,Satisfaction rating,_Satisfa��o,Unnamed: 16,Unnamed: 17,N,Unnamed: 19
0,ID0006,01/01/2015 11:57,Becky,Fridge,N,N,NaN,NaN,NaN,NaN,N,N,-1,-1,-1,Sem Avalia��o,NaN,NaN,NaN,NaN
1,ID0014,02/01/2015 13:45,Becky,Air Conditioner,N,N,NaN,NaN,NaN,NaN,N,N,-1,-1,-1,Sem Avalia��o,NaN,NaN,NaN,NaN
2,ID0027,04/01/2015 14:49,Becky,Air Conditioner,N,N,NaN,NaN,NaN,NaN,N,N,-1,-1,-1,Sem Avalia��o,NaN,NaN,NaN,NaN
3,ID0034,06/01/2015 11:12,Becky,Television,N,N,NaN,NaN,NaN,NaN,N,N,-1,-1,-1,Sem Avalia��o,NaN,NaN,NaN,NaN
4,ID0036,06/01/2015 17:21,Becky,Washing Machine,N,N,NaN,NaN,NaN,NaN,N,N,-1,-1,-1,Sem Avalia��o,NaN,NaN,NaN,NaN


In [5]:
pd.set_option('display.max_rows', None)
df.dtypes

Call Id                  object
Date                     object
Agent                    object
Department               object
_Answered (Y/N)          object
_Resolved                object
_Speed of Answer        float64
_AvgTalkDuration        float64
_Satisfaction rating    float64
dur_aux                 float64
Answered (Y/N)           object
Resolved                 object
Speed of Answer           int64
AvgTalkDuration           int64
Satisfaction rating       int64
_Satisfa��o              object
Unnamed: 16             float64
Unnamed: 17             float64
N                       float64
Unnamed: 19             float64
dtype: object

In [6]:
# Criando a coluna de data
df['data'] = df['Date'].apply(lambda x: x.split(' ')[0])

In [7]:
df['dur_aux'].fillna(0, inplace=True)

C:\Users\Romulo Vieira\AppData\Local\Temp\ipykernel_21840\1692652733.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['dur_aux'].fillna(0, inplace=True)


In [8]:
# Modificando os dados
# Transformando a coluna DATA em fomrato datetime
# Retirando as vírgulas da coluna dur_aux e colocando pontos no lugar
# Transformando a coluna dur_aux em float

df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')
df['dur_aux'] = df['dur_aux'].str.replace(',', '.')
df['dur_aux'] = df['dur_aux'].astype('float64')


AttributeError: Can only use .str accessor with string values!

In [9]:
df['Satisfaction rating'].value_counts(dropna=False)

Satisfaction rating
-1    499
Name: count, dtype: int64

In [10]:
# Retirando os -1 da coluna de Satisfação e colocando NaN no lugar (Not a Number)

df_sat = df.replace({'Satisfaction rating':-1}, np.nan)
df_sat['Satisfaction rating'].value_counts(dropna=False)

Satisfaction rating
NaN    499
Name: count, dtype: int64

In [11]:
# Sumarizando as colunas por data, usando diferentes agregadores

df_plot = df_sat[['data', 'Satisfaction rating', 'Call Id', 'Speed of Answer', 'dur_aux']].groupby('data').agg(
  {'Satisfaction rating':'mean',
  'Call Id':'count',
  'Speed of Answer':'sum',
  'dur_aux':'sum'}).reset_index()


In [12]:
# Criando indicadores

df_plot['TM espera'] = df_plot['Speed of Answer'] / df_plot['Call Id']
df_plot['TM atendimento'] = df_plot['dur_aux'] / df_plot['Call Id']

In [13]:
df_plot.head()

,data,Satisfaction rating,Call Id,Speed of Answer,dur_aux,TM espera,TM atendimento
0,2015-01-01,NaN,7,-7,0.0,-1.0,0.0
1,2015-01-02,NaN,7,-7,0.0,-1.0,0.0
2,2015-01-03,NaN,5,-5,0.0,-1.0,0.0
3,2015-01-04,NaN,6,-6,0.0,-1.0,0.0
4,2015-01-05,NaN,2,-2,0.0,-1.0,0.0


In [14]:
df_plot.rename(columns={'Satisfaction rating':'satisfacao_cliente', 'Call Id':'total_ligacoes',
           'Speed of Answer':'tempo_para_atendimento', 'dur_aux':'duracao_atendimento'}, inplace=True)

In [15]:
df_plot.head()

,data,satisfacao_cliente,total_ligacoes,tempo_para_atendimento,duracao_atendimento,TM espera,TM atendimento
0,2015-01-01,NaN,7,-7,0.0,-1.0,0.0
1,2015-01-02,NaN,7,-7,0.0,-1.0,0.0
2,2015-01-03,NaN,5,-5,0.0,-1.0,0.0
3,2015-01-04,NaN,6,-6,0.0,-1.0,0.0
4,2015-01-05,NaN,2,-2,0.0,-1.0,0.0


In [16]:
fig = px.scatter(data_frame=df_plot, x='TM espera', y='satisfacao_cliente', trendline='ols', trendline_color_override='orange',
     title='Variação da Satisfação por Tempo de Espera', height=600)
fig.update_layout(xaxis_title='Tempo Médio de Espera', yaxis_title='Satisfação do Cliente')
fig.update_traces(marker={'size':8, 'line':{'width':1, 'color':'black'}}, line={'width':4})
fig.show()

In [17]:
df_plot.head()

,data,satisfacao_cliente,total_ligacoes,tempo_para_atendimento,duracao_atendimento,TM espera,TM atendimento
0,2015-01-01,NaN,7,-7,0.0,-1.0,0.0
1,2015-01-02,NaN,7,-7,0.0,-1.0,0.0
2,2015-01-03,NaN,5,-5,0.0,-1.0,0.0
3,2015-01-04,NaN,6,-6,0.0,-1.0,0.0
4,2015-01-05,NaN,2,-2,0.0,-1.0,0.0


In [18]:
px.scatter(data_frame=df_plot, x='total_ligacoes', y='satisfacao_cliente', width=600, height=500,
           trendline_color_override='orange', trendline='ols',
           title='Variação da Satisfação por Total de Ligações')

In [19]:
px.imshow(df_plot.corr(), zmin=-1, zmax=1, width=600, height=500,
          title='Mapa de Correlação entre as Variáveis')

In [20]:
fig = px.imshow(df_plot.corr(), zmin=-1, zmax=1, width=600, height=500, text_auto=True,
          title='Mapa de Correlação entre as Variáveis')
fig.show()

In [21]:
fig = px.imshow(df_plot.corr(), zmin=-1, zmax=1, width=600, height=500, text_auto='.2f')
fig.update_xaxes(tickangle=45)
fig.show()

In [22]:
fig = px.imshow(df_plot.corr(), zmin=-1, zmax=1, width=600, height=500, text_auto='.2f', color_continuous_scale=['red', 'yellow', 'blue'])
fig.update_xaxes(tickangle=45)
fig.show()

In [23]:
px.line(data_frame=df_plot, x='data', y='satisfacao_cliente')

In [24]:
px.line(data_frame=df_plot, x='data', y='satisfacao_cliente', hover_data='total_ligacoes')

In [25]:
px.bar(data_frame=df_plot, x='data', y='total_ligacoes')

In [26]:
px.bar(data_frame=df_plot, x='data', y='total_ligacoes', hover_data='satisfacao_cliente')

In [27]:
from plotly.subplots import make_subplots

fig_line = px.line(data_frame=df_plot, x='data', y='satisfacao_cliente', hover_data='total_ligacoes')
fig_bar = px.bar(data_frame=df_plot, x='data', y='total_ligacoes', hover_data='satisfacao_cliente', color_discrete_sequence=['orange'])
fig = make_subplots(rows=2, cols=1, subplot_titles=('Satisfação', 'Total de Ligações'), shared_xaxes=True, vertical_spacing=0.1)

for x in fig_line['data']:
  fig.add_trace(x, row=1, col=1)
 
for figo in fig_bar['data']:
  fig.add_trace(figo, row=2, col=1)

fig.update_layout(height=600)
fig.show()